### **YOLO v1 pytorch implementation**

**Clone repository**

In [ ]:
!git clone https://github.com/godwinrayanc/YOLOv1-Pytorch.git
%cd /content/YOLOv1-Pytorch/pytorch-yolov1-trainval

Cloning into 'YOLOv1-Pytorch'...
remote: Enumerating objects: 119, done.
remote: Counting objects: 100% (119/119), done.
remote: Compressing objects: 100% (114/114), done.
remote: Total 119 (delta 35), reused 14 (delta 1), pack-reused 0
Receiving objects: 100% (119/119), 1.57 MiB | 9.68 MiB/s, done.
Resolving deltas: 100% (35/35), done.
/content/YOLOv1-Pytorch/pytorch-yolov1-trainval


**Importing stuff**

Download the 4Gb archive.zip file from [here](https://https://www.kaggle.com/dataset/734b7bcb7ef13a045cbdd007a3c19874c2586ed0b02b4afc86126e89d00af8d2)

upload archive.zip to your drive and then unzip it using

In [ ]:
!unzip -q -u "/content/drive/MyDrive/archive.zip" -d "/content/YOLOv1-Pytorch/pytorch-yolov1-trainval/data"

/content/drive/MyDrive/yolo


### **Train properly**

**Import dependencies**

In [ ]:
import math
import csv
import torch
import torchvision.transforms as transforms
import torch.optim as optim
import torchvision.transforms.functional as FT
from tqdm.notebook import tqdm
from torch.utils.data import DataLoader
from backbone import resnet50, vgg16
from dataset import VOCDataset
from pathlib import Path
from utils import (
    non_max_suppression,
    mean_average_precision,
    intersection_over_union,
    cellboxes_to_boxes,
    get_bboxes,
    plot_image,
    save_checkpoint,
    load_checkpoint,
)
from loss import YoloLoss
import datatrans

seed = 123
torch.manual_seed(seed)

**Set the hyperparameter values. Make sure to edit the path for saving the trained model**

In [ ]:

init_lr = 0.00002
base_lr = 0.00008
DEVICE = "cuda" if torch.cuda.is_available else "cpu"
BATCH_SIZE = 32
WEIGHT_DECAY = 5e-4
EPOCHS = 40
NUM_WORKERS = 4
PIN_MEMORY = True
SAVE_PATH = "resnet50.pth"
IMG_DIR = "data/images"
LABEL_DIR = "data/labels"

**Laerning rate schedule implementation**

In [ ]:

def update_lr(optimizer, epoch):
    if epoch == 0:
        lr = init_lr 
    elif epoch == 2:
        lr = base_lr
    elif epoch == 15:
        lr = 0.00004
    elif epoch == 25:
        lr = 0.00002
    else:
        return

    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group['lr']

**Train function**

In [ ]:
def train_fn(train_loader, model, optimizer, loss_fn, epoch):
    loop = tqdm(train_loader, leave=True)
    mean_loss = []

    for batch_idx, (x, y) in enumerate(loop):
        #Update learning rate.
        update_lr(optimizer, epoch)
        lr = get_lr(optimizer)

        x, y = x.to(DEVICE), y.to(DEVICE)
        out = model(x)
        loss = loss_fn(out, y)
        mean_loss.append(loss.item())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # update progress bar
        loop.set_postfix(loss=loss.item())
        torch.cuda.empty_cache()

    print(f"Mean loss was {sum(mean_loss)/len(mean_loss)}")

**Main function**

In [ ]:
def main():

    #Choose model from backbone.py
    model = resnet50(split_size=7, num_boxes=2, num_classes=20, pretrained = True).to(DEVICE)
    print('Number of model parameters: {}'.format(sum([p.data.nelement() for p in resnet50.parameters()])))

    optimizer = optim.Adam(model.parameters(), lr=init_lr, weight_decay=WEIGHT_DECAY)

    loss_fn = YoloLoss()

    #Load data into data loader
    train_dataset = VOCDataset(
        "data/train.csv",
        transform=transform,
        img_dir=IMG_DIR,
        label_dir=LABEL_DIR)

    val_dataset = VOCDataset(
        "data/validation.csv", 
        transform=transform_test, 
        img_dir=IMG_DIR, 
        label_dir=LABEL_DIR)

    train_loader = DataLoader(
        dataset=train_dataset,
        batch_size=BATCH_SIZE,
        num_workers=NUM_WORKERS,
        pin_memory=PIN_MEMORY,
        shuffle=True,
        drop_last=True,
    )

    val_loader = DataLoader(
        dataset=val_dataset,
        batch_size=BATCH_SIZE,
        num_workers=NUM_WORKERS,
        pin_memory=PIN_MEMORY,
        shuffle=True,
        drop_last=True,
    )


    #Create empty list to store accuracy values to plot later
    acc_list = []
    best_mAP = 0

    #Start counting time
    import time
    start_full_time = time.time()

    #Start training Loop
    for epoch in range(EPOCHS):
        start_time = time.time() 
        #Calculate Training mAP
        pred_boxes, target_boxes = get_bboxes(train_loader, model, iou_threshold=0.5, threshold=0.4)
        train_mAP = mean_average_precision(pred_boxes, target_boxes, 
                                  iou_threshold=0.5, box_format="midpoint")
        torch.cuda.empty_cache()
        #Calculate Validation mAP
        pred_boxes, target_boxes = get_bboxes(val_loader, model, iou_threshold=0.5, threshold=0.4)
        val_mAP = mean_average_precision(pred_boxes, target_boxes, 
                                  iou_threshold=0.5, box_format="midpoint")
        torch.cuda.empty_cache()
        print(f"Epoch: {epoch}" + " |" f"Train mAP: {train_mAP}" + " |" + f"Validation mAP: {val_mAP}")
        
        
        #write losses to csv file
        acc_list.append([epoch, train_mAP, val_mAP])
        with open("resnet50.csv", 'a') as f:
          writer = csv.writer(f)
          writer.writerow([epoch, train_mAP, val_mAP])
        

        #Update best mAP value
        if val_mAP > 0.40 and val_mAP > best_mAP:
            best_mAP = val_mAP
            filename = str(epoch)+ SAVE_PATH
            torch.save(model.state_dict(), filename)
            
            time.sleep(10)
        
        #Switch to training
        model.train()

        #Train network
        train_fn(train_loader, model, optimizer, loss_fn, epoch)
        torch.cuda.empty_cache()

        #Demarcation
        print(f"time: {time.time() - start_time}")
        print("----------------------------------------")
        print("----------------------------------------")


    print(f"Total_Time: {time.time() - start_full_time}")   
        


if __name__ == "__main__":
    main()